In [1]:
# 数据集
def loadDataSet():
    postingList = [['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
                  ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                  ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
                  ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                  ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
                  ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
    classVec = [0, 1, 0, 1, 0, 1]  # 1代表侮辱性文字，0代表正常言论
    return postingList, classVec

# 创建不重复数据集词列表
def createVocabList(dataSet):
    vocabSet = set([])
    for document in dataSet:
        vocabSet = vocabSet | set(document)  # | 表示并集
    return list(vocabSet)

# 返回输入数据的词向量
def setOfWords2Vec(vocabList, inputSet):
    returnVec = [0] * len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] = 1
        else:
            print('the word: %s is not in my Vocabulary!' % word)
    return returnVec

In [2]:
listOPosts, listClasses = loadDataSet()
myVocabList = createVocabList(listOPosts)
myVocabList

['posting',
 'mr',
 'worthless',
 'dalmation',
 'stop',
 'maybe',
 'garbage',
 'how',
 'to',
 'cute',
 'stupid',
 'problems',
 'is',
 'my',
 'quit',
 'help',
 'so',
 'take',
 'buying',
 'love',
 'steak',
 'please',
 'him',
 'dog',
 'food',
 'ate',
 'licks',
 'I',
 'not',
 'flea',
 'has',
 'park']

In [3]:
setOfWords2Vec(myVocabList, listOPosts[0])

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0]

In [4]:
setOfWords2Vec(myVocabList, listOPosts[3])

[1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

## 训练算法

In [5]:
from numpy import *

In [6]:
# 参数：词向量矩阵，言论类别标签
# 返回值：每个非侮辱性词汇概率，每个侮辱性词汇概率，侮辱性言论概率
def trainNB0(trainMatrix, trainCategory):
    numTrainDocs = len(trainMatrix)  # 言论条数
    numWords = len(trainMatrix[0])  # 单条词向量数量
    pAbusive = sum(trainCategory) / float(numTrainDocs)  # 侮辱性言论出现的概率
    
    p0Num = ones(numWords)
    p1Num = ones(numWords)  # 每个侮辱性词汇出现的次数（初始化为1防止概率为0）
    p0Denom = 2.0
    p1Denom = 2.0  # 侮辱性词汇出现的总次数
    
    for i in range(numTrainDocs):
        if trainCategory[i] == 1:
            p1Num += trainMatrix[i]
            p1Denom += sum(trainMatrix[i])
        else:
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])
    p1Vect = log(p1Num / p1Denom)  # 每个侮辱性词汇出现的概率（加log防止下溢出）
    p0Vect = log(p0Num / p0Denom)
    return p0Vect, p1Vect, pAbusive

In [7]:
listOPosts, listClasses = loadDataSet()
myVocabList = createVocabList(listOPosts)

trainMat = []  # 词向量矩阵
for postinDoc in listOPosts:
    trainMat.append(setOfWords2Vec(myVocabList, postinDoc))

In [8]:
p0V, p1V, pAb = trainNB0(trainMat, listClasses)
p0V, p1V, pAb

(array([-3.25809654, -2.56494936, -3.25809654, -2.56494936, -2.56494936,
        -3.25809654, -3.25809654, -2.56494936, -2.56494936, -2.56494936,
        -3.25809654, -2.56494936, -2.56494936, -1.87180218, -3.25809654,
        -2.56494936, -2.56494936, -3.25809654, -3.25809654, -2.56494936,
        -2.56494936, -2.56494936, -2.15948425, -2.56494936, -3.25809654,
        -2.56494936, -2.56494936, -2.56494936, -3.25809654, -2.56494936,
        -2.56494936, -3.25809654]),
 array([-2.35137526, -3.04452244, -1.94591015, -3.04452244, -2.35137526,
        -2.35137526, -2.35137526, -3.04452244, -2.35137526, -3.04452244,
        -1.65822808, -3.04452244, -3.04452244, -3.04452244, -2.35137526,
        -3.04452244, -3.04452244, -2.35137526, -2.35137526, -3.04452244,
        -3.04452244, -3.04452244, -2.35137526, -1.94591015, -2.35137526,
        -3.04452244, -3.04452244, -3.04452244, -2.35137526, -3.04452244,
        -3.04452244, -2.35137526]),
 0.5)

## 测试算法

In [9]:
def classifyNB(vec2Classify, p0Vec, p1Vec, pClass1):
    p1 = sum(vec2Classify * p1Vec) + log(pClass1)
    p0 = sum(vec2Classify * p0Vec) + log(1.0 - pClass1)
    if p1 > p0:
        return 1
    else:
        return 0
    
def testingNB():
    listOPosts, listClasses = loadDataSet()
    myVocabList = createVocabList(listOPosts)
    trainMat = []
    for postinDoc in listOPosts:
        trainMat.append(setOfWords2Vec(myVocabList, postinDoc))
    p0V, p1V, pAb = trainNB0(array(trainMat), array(listClasses))
    
    testEntry = ['love', 'my', 'dalmation']
    thisDoc = array(setOfWords2Vec(myVocabList, testEntry))
    print(testEntry, 'classified as: ', classifyNB(thisDoc, p0V, p1V, pAb))
    
    testEntry = ['stupid', 'garbage']
    thisDoc = array(setOfWords2Vec(myVocabList, testEntry))
    print(testEntry, 'classified as: ', classifyNB(thisDoc, p0V, p1V, pAb))

In [10]:
testingNB()

['love', 'my', 'dalmation'] classified as:  0
['stupid', 'garbage'] classified as:  1


In [11]:
# 词袋模型
def bagOfWordsVecMN(vocabList, inputSet):
    returnVec = [0] * len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] += 1
    return returnVec

## 实例：过滤垃圾邮件

In [57]:
def textParse(bigString):
    import re
    listOfTokens = re.split(r'\W+', bigString)
    return [tok.lower() for tok in listOfTokens if len(tok) > 2]

In [58]:
bigString = open('email/spam/1.txt').read()
bigString

'--- Codeine 15mg -- 30 for $203.70 -- VISA Only!!! --\n\n-- Codeine (Methylmorphine) is a narcotic (opioid) pain reliever\n-- We have 15mg & 30mg pills -- 30/15mg for $203.70 - 60/15mg for $385.80 - 90/15mg for $562.50 -- VISA Only!!! ---'

In [59]:
textParse(bigString)

['codeine',
 '15mg',
 'for',
 '203',
 'visa',
 'only',
 'codeine',
 'methylmorphine',
 'narcotic',
 'opioid',
 'pain',
 'reliever',
 'have',
 '15mg',
 '30mg',
 'pills',
 '15mg',
 'for',
 '203',
 '15mg',
 'for',
 '385',
 '15mg',
 'for',
 '562',
 'visa',
 'only']

In [99]:
def spamTest():
    docList = []  # 存储每封邮件的词列表集合
    classList = []  # 邮件类别标签集合
    fullText = []  # 所有邮件的词的集合
    for i in range(1, 26):
        wordList = textParse(open('email/spam/%d.txt' % i).read())
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(1)
        
        wordList = textParse(open('email/ham/%d.txt' % i).read())
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(0)
        
    vocabList = createVocabList(docList)
    trainingSet = list(range(50))  # 存储标号的训练集
    testSet = []  # 存储标号的测试集
    
    for i in range(10):  # 随机将10个样例加入测试集
        randIndex = int(random.uniform(0, len(trainingSet)))  # 返回区间随机数
        testSet.append(trainingSet[randIndex])
        del trainingSet[randIndex]
    
    trainMat = []
    trainClasses = []
    for docIndex in trainingSet:  # 构建训练集词向量矩阵
        trainMat.append(setOfWords2Vec(vocabList, docList[docIndex]))
        trainClasses.append(classList[docIndex])
        
    p0V, p1V, pSpam = trainNB0(array(trainMat), array(trainClasses))
    
    errorCount = 0
    for docIndex in testSet:
        wordVector = setOfWords2Vec(vocabList, docList[docIndex])
        if classifyNB(array(wordVector), p0V, p1V, pSpam) != classList[docIndex]:
            errorCount += 1
    print('the error rate is: ', float(errorCount) / len(testSet))

In [100]:
spamTest()

the error rate is:  0.1
